In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

In [2]:
movies = pd.read_csv("movies_metadata.csv")
print(movies.columns)
movies.head()

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


C:\Users\uSER\AppData\Local\Temp\ipykernel_59312\555436180.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [4]:
keywords = pd.read_csv('keywords.csv')

In [5]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

movies['id'] = movies['id'].apply(clean_ids)
movies = movies[movies['id'].notnull()]

In [6]:
movies['id'] = movies['id'].astype('int')
keywords['id'] = keywords['id'].astype('int')

movies = movies.merge(keywords, on='id')

# movies.head()

In [7]:
movies["keywords"] = movies["keywords"].apply(literal_eval)

In [8]:
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 10:
            names = names[:10]
        return names
    return []

movies['keywords'] = movies['keywords'].apply(generate_list)
movies['genres'] = movies['genres'].apply(lambda x: x[:10])

movies[['title', 'keywords', 'genres']].head()

,title,keywords,genres
0,Toy Story,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]"
1,Jumanji,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]"
3,Waiting to Exhale,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[baby, midlife crisis, confidence, aging, daug...",[Comedy]


In [9]:
def sanitize(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ','')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

for feature in ['genres', 'keywords']:
    movies[feature] = movies[feature].apply(sanitize)

In [10]:
def movie_soup(x):
    return  x["title"] + " " + " ".join(x['genres']) + " "+x['overview']+" "+" ".join(x['keywords'])

movies['overview'] = movies['overview'].fillna('')
movies['title'] = movies['title'].fillna('')
movies['soup'] = movies.apply(movie_soup, axis=1)

In [11]:
books = pd.read_csv("top2k_book_descriptions.csv", index_col=0)
print(books.columns)
# books.head()

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url', 'goodreads_book_id', 'tag_name', 'description'],
      dtype='object')


In [12]:
books['tag_name'] = books['tag_name'].apply(lambda x: literal_eval(x) if literal_eval(x) else np.nan)
books = books[books['description'].notnull() | books['tag_name'].notnull()]
books = books.fillna('')

In [13]:
def book_soup(x):
    soup = x["original_title"]+" "+x["description"]+" "+" ".join(x['tag_name'])+" "+x["authors"]
    return soup

In [14]:
books["soup"] = books.apply(book_soup, axis=1)


In [22]:
soups = pd.concat([books['soup'],movies['soup']],ignore_index=True)

In [23]:
count = CountVectorizer(stop_words = "english")
count.fit(soups)

books_matrix = count.transform(books['soup'])
movies_matrix = count.transform(movies['soup'])


books_matrix.shape, movies_matrix.shape

((1987, 98296), (46482, 98296))

In [24]:
cosine_sim = cosine_similarity(books_matrix, movies_matrix)

In [25]:
books = books.reset_index()
indices = pd.Series(books.index, index=books['original_title'].apply(lambda x: x.lower() if x is not np.nan else "")).drop_duplicates()

In [27]:
def content_recommender(title):
    idx = indices[title.lower()]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    sim_scores = sim_scores[:10]

    # book_indices = [i[0] for i in sim_scores]
    movie_indices = [i[0] for i in sim_scores]

    # return books.iloc[book_indices]
    return movies.iloc[movie_indices]

In [28]:
!pip3 install -q ipywidgets
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [31]:
import ipywidgets
from IPython.display import HTML
def showhtml(recommendations):
    html = ' '.join([f"""
     <div class="flip-card">
      <div class="flip-card-inner">
        <div class="flip-card-front">
          <img src="{recommendations.iloc[i]['image_url']}" alt="Avatar" style="width:300px;height:300px;">
        </div>
        <div class="flip-card-back">
          <h4>{recommendations.iloc[i]['title']}</h4>
          <p>by {recommendations.iloc[i]['authors']}</p>
        </div>
      </div>
    </div> """ for i in range(10)])
    html = "<div class='grid'>"+html+"</div>"
    html +="""<style>
    .flip-card {
      background-color: transparent;
      width: 200px;
      height: 300px;
      border: 1px solid #f1f1f1;
    }

    .flip-card-inner {
      position: relative;
      width: 100%;
      height: 100%;
      text-align: center;
      transition: transform 0.8s;
      transform-style: preserve-3d;
    }

    .flip-card:hover .flip-card-inner {
      transform: rotateY(180deg);
    }

    .flip-card-front, .flip-card-back {
      position: absolute;
      width: 100%;
      height: 100%;
      -webkit-backface-visibility: hidden; /* Safari */
      backface-visibility: hidden;
    }

    .flip-card-front {
      background-color: #bbb;
      color: black;
    }

    .flip-card-back {
    padding:10px;
      background-color: dodgerblue;
      color: white;
      transform: rotateY(180deg);
    }
    .grid {
        display: grid;
        grid-template-columns: 30% 30% 30%;
        grid-template-rows: 25% 25% 25%;
        grid-gap: 5%;
    }
    </style>"""
    return html


def show_movies(book_name='The History of Love'):
    recommendations = content_recommender(book_name)
    for i in range(10):
#         disPic(recommendations["image_url"].iloc[i])
          print(recommendations["original_title"].iloc[i])
        # print("Description: " + recommendations["description"].iloc[i])
    # display(HTML(showhtml(recommendations)))
display(ipywidgets.interact(show_movies))

interactive(children=(Text(value='The History of Love', description='book_name'), Output()), _dom_classes=('wi…

<function __main__.show_movies(book_name='The History of Love')>